In [2]:
import tensorflow as tf
import numpy as np
from Transformer import Transformer

In [62]:
eng_path = "v2/en-te/train.en"
tel_path = "v2/en-te/train.te"
with open(eng_path, 'r') as file:
    eng_data = file.readlines()
with open(tel_path, 'r', encoding='UTF-8') as file:
    tel_data = file.readlines()

In [63]:
len(tel_data), len(eng_data)

(4946035, 4946035)

In [64]:
eng_data[0], tel_data[0]

('Have you heard about Foie gras?\n',
 'ఇక ఫ్రూట్ ఫ్లైస్ గురించి మీరు విన్నారా?\n')

In [73]:
tel_data = [x.strip('\n') for x in tel_data]
eng_data = [x.strip('\n') for x in eng_data]

In [74]:
d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
max_sequence_length = 200
drop_prob = 0.1
num_layers = 2
START_TOKEN = ''
PADDING_TOKEN = ''
END_TOKEN = ''

In [75]:
tel_uni = set()
for sentence in tel_data:
    sentence = sentence.lower()
    for char in sentence:
        tel_uni.add(char)

eng_uni = set()
for sentence in eng_data:
    sentence = sentence.lower()
    for char in sentence:
        eng_uni.add(char)

tel_uni = list(tel_uni) + [START_TOKEN, END_TOKEN, PADDING_TOKEN]
eng_uni = list(eng_uni) + [START_TOKEN, END_TOKEN, PADDING_TOKEN]

In [ ]:
tel_invalid = ['ತ', 'ḷ', 'т', 'ि', 'ल', 'ś', 'メ', 'ਅ', '✺', '☘', 'ग', 'j', 'я', 'র', 'φ', 'ध', 'س', '◘', 'ń', 'µ', '¡', 'ت', 'ா', 'i', 'د', 'ک', '»', '\xad',
               'ശ', 'া', 'ਤ', '©', '𐎼', '情', 'р', 'è', '毒', 'к', 'ম', 'ے', 'ب', 'n', '➦', 's', 'e', 'ਰ', 'ਲ', 'ä', '✶', '\uf0fc', 'ক', '•', 'र', 'g', '❂',
               '∠', 'ঘ', '▾', 'ع', 'ே', 'भ', 'ς', 'ḍ', '្', 'च', '❋', 'ո', 'হ', 'ն', 'ವ', 'व', 'ه', 'ರ', '份', 'ਾ', '¤', 'γ', 'շ', 'π', 'অ', 'ի', 'v', 'ற',
               '早', 'न', 'ó', 'q', 'ज', 'خ', '𐎥', 'ে', '∘', 'श', '\u2060', '\u200b', 'ೂ', '£', '্', 'ு', '►', 'ُ', '✧', '△', 'জ',
               '⇔', 'α', 'h', '❏', 'چ', 'ত', 'կ', 'つ', '→', '\u200c', '←', '𐎭', '✪', 'گ', 'आ', '«', 'ك', 'ç', '‘', '\ufeff', 'प', 'य', 'd', 'ਗ',
               'æ', '♦', 'ア', '\u200d', '।', 'ា', '§','ह', 'м', 'ニ', '□', 'त', '♯', 'σ', 'տ', 'z', 'ւ', 'ர', '്', 'θ', 'ி',
               '▪', 'ئ', 'ा', 'ೇ', 'ব', '✦', 'ਕ','ी', '≈', 'ن', '😊', '∇', '“', 'क', 'ج', 'и', 'ಡ', 'յ', '♠', 'ց', 'ο', 'մ', 'ଅ', ']',
               'ô', '_', 'য', '戀', '😍', '愛', 'उ', 'ρ', '№', 'ā', 'ة', 'ύ', 'գ', '身', 'ब', '❑', 'w', 'x', 'c', 'ন', 'в', '⦾', 'ش', 'ز', 'غ',  'y',
               'm', '🥰', '◆', 'ধ', 'ञ', '恋', 'ե', 'ത', 'հ', '\u200e', '❤', 'š', 'е', 'β', 'ஆ', '♪', 'द', 'ण', 'ര', 'ù', 'ν', 'b', 'े', 'å', 'ौ',
               '★', '∑', 'a', 'ս', 'ದ', 'េ', 'é', 'স', '\uf0d8', '्', 'ន', 'ϙ', 't', '¸', 'ಮ', 'ا', 'ł', '`', 'ಕ', 'দ', 'ఽ', 'ø', 'í', 'ស', 'ę', '⇒', 'r', 'ਖ',
               '₹', 'म', 'ھ', '\u2061', 'ր', 'ਮ', '਼', 'ہ', 'ć', 'դ', 'ਝ', 'ò', 'ي', 'ö', 'ω', 'δ', 'k', 'ل', 'வ', '♥', '⊕', '\u202a', 'و', 'թ', 'پ', 'க', 'ទ',
               'ε', '—', 'ϝ', '➥', 'о', 'p', '\uf02a', 'œ', '\xa0', '×', 'ಗ', 'ñ', 'ល', 'ǧ', 'ա', 'û', 'у', 'ī', 'ប', 'μ', 'с', 'ú', '≤', '緣', '²',
               '⁄', 'ی', 'u', 'स', '✹', '❖', 'ਘ', 'պ', '॥', 'ল', 'f', '𐎠', '⍟', 'م', 'ں', 'ر', 'も','焉','ズ', 'ĕ','龜', '౭','~','а', 'ē', '<',
              'パ', '🙂','ோ','κ', '¼', '日','信', 'п', 'थ', 'ǵ','â', '³','✓','ض','‡', '>', '^', 'イ','₂','o','౮','º','º','本','₣', 'ಶ', 'א', 'अ', 'ो',
              'н', '话','ב', 'ನ', 'ί','ы', '6', '8', 'ந','ְ', 'ْ', 'ê','ר','д', 'ஏ','ם', '⇨', '\\','ड','€', '❉', 'ж','¿', '′','َ', '✯', '●', '➢', 'த', 'ट', 'ப', 'č',
              'ë', 'ம', '茲','л', 'з','电','✭','η','ņ','耆', '½','ष','л', 'з','电','✭','3', 'ல',  '➤','ய', 'ι','™', '®', '|', '○','ؤ','ה','ओ']

tel_uni_copy = tel_uni.copy()  
for char in tel_uni_copy:
    if char in tel_invalid:
        tel_uni.remove(char)

In [94]:
print(tel_uni)

['ř', 'థ', 'అ', 'ು', '7', '*', 'న', '-', 'ఁ', 'ఱ', '˚', 'ఏ', 'ָ', '\u200f', 'ా', 'ఈ', 'l', 'ఔ', 'ఋ', 'ద', '’', '0', 'ౖ', 'ృ', 'ఙ', '=', 'బ', '{', 'ొ', '?', 'ర', 'ఛ', 'ప', 'ౌ', 'ِ', '¯', 'ష', 'ಜ', '⚹', '‑', 'వ', 'リ', 'ಚ', '\u0c11', 'ధ', '&', 'ु', 'ి', '°', '/', 'ఓ', 'య', '…', ',', 'ఝ', 'ట', '´', '!', '∙', '∼', '್', '”', 'ో', '1', '–', '#', 'ళ', 'ం', 'క', 'మ', '़', 'ణ', 'ఫ', 'ూ', 'ః', '@', 'డ', 'శ', 'ీ', '\uf0a7', 'ַ', '+', 'త', 'భ', ':', '⦁', 'ఉ', 'ఒ', '்', 'ं', '4', 'ై', '[', 'ఐ', ' ', 'ఘ', '౦', '"', 'చ', '·', 'ఢ', 'జ', 'ఇ', '%', '}', 'ఖ', 'ఞ', "'", '౬', '్', 'ఆ', '−', 'ె', 'ఎ', 'ಾ', 'ు', 'ृ', 'ँ', ')', 'ల', '9', 'స', '.', '$', '౧', '(', '2', 'హ', 'ʹ', 'ే', 'ఊ', 'ఠ', 'గ', '5', '', '', '']


In [95]:
tel_to_ix = {ch: i for i, ch in enumerate(tel_uni)}
ix_to_tel = {i: ch for i, ch in enumerate(tel_uni)}
eng_to_ix = {ch: i for i, ch in enumerate(eng_uni)}
ix_to_eng = {i: ch for i, ch in enumerate(eng_uni)}

In [96]:
def is_valid_tokens(sentence, vocab):
    for char in list(set(sentence)):
        if char not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(sentence) < max_sequence_length-1

valid_sentence_indices = []
for index in range(len(tel_data)):
    tel_sen, eng_sen = tel_data[index], eng_data[index]
    if is_valid_length(tel_sen, max_sequence_length)  \
        and is_valid_length(eng_sen, max_sequence_length) \
        and is_valid_tokens(tel_sen, tel_uni):
        valid_sentence_indices.append(index)

In [97]:
tel_sens = [tel_data[i] for i in valid_sentence_indices]
eng_sens = [eng_data[i] for i in valid_sentence_indices]

In [100]:
class TextDataset(tf.keras.utils.Sequence):
    def __init__(self, english_sentences, kannada_sentences, batch_size):
        self.english_sentences = english_sentences
        self.kannada_sentences = kannada_sentences
        self.batch_size = batch_size

    def __len__(self):
        return len(self.english_sentences) // self.batch_size

    def __getitem__(self, idx):
        start_index = idx * self.batch_size
        end_index = start_index + self.batch_size
        english_batch = self.english_sentences[start_index:end_index]
        kannada_batch = self.kannada_sentences[start_index:end_index]
        return english_batch, kannada_batch

In [101]:
td = TextDataset(eng_sens, tel_sens, batch_size)

In [22]:
NEG_INFTY = -1e9

def create_mask(eng_batch, tel_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = np.full((200,200), True)
    look_ahead_mask = np.triu(look_ahead_mask, k=1)
    encoder_padding_mask = np.full((num_sentences, 200, 200), False)
    decoder_padding_mask_self_attention = np.full((num_sentences, 200, 200), False)
    decoder_padding_mask_cross_attention = np.full((num_sentences, 200, 200), False)

    for idx in range(num_sentences):
        eng_sen_len, tel_sen_len = len(eng_batch[idx]), len(tel_batch[idx])
        eng_chars_to_padding_mask = np.arange(eng_sen_len + 1, 200)
        tel_chars_to_padding_mask = np.arange(tel_sen_len + 1, 200)
        encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
        encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
        decoder_padding_mask_self_attention[idx, :, tel_chars_to_padding_mask] = True
        decoder_padding_mask_self_attention[idx, tel_chars_to_padding_mask, :] = True
        decoder_padding_mask_cross_attention[idx, :, tel_chars_to_padding_mask] = True
        decoder_padding_mask_cross_attention[idx, tel_chars_to_padding_mask, :] = True

    encoder_padding_mask = np.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_padding_mask_self_attention = np.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_padding_mask_cross_attention = np.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)

    return encoder_padding_mask, decoder_padding_mask_self_attention, decoder_padding_mask_cross_attention

In [123]:
transformer = Transformer(d_model, num_heads, ffn_hidden, drop_prob, num_layers, max_sequence_length,len(tel_uni),
                          eng_to_ix, tel_to_ix, START_TOKEN, END_TOKEN, PADDING_TOKEN)

In [24]:
criterion = tf.keras.losses.SparseCategoricalCrossentropy(ignore_class=tel_to_ix[PADDING_TOKEN], reduction='none')

initializer = tf.keras.initializers.GlorotUniform()
for params in transformer.trainable_variables:
    if len(params.shape) > 1:
        initializer(params)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

In [25]:
def calculate_loss(predictions, labels, padding_token_index):
    mask = tf.math.logical_not(tf.equal(labels, padding_token_index))
    loss = criterion(labels, predictions, sample_weight=tf.cast(mask, dtype=tf.float32))
    return tf.reduce_mean(loss)

In [26]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [131]:
total_loss = 0
epochs = 6

with tf.device('/GPU:0'):

  for epoch in range(epochs):
      print("Epoch", epoch)
      for batch_num, batch in enumerate(td):
          eng_bat, tel_bat = batch
          eng_bat = [x.lower() for x in eng_bat]

          with tf.GradientTape() as tape:
              encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_mask(eng_bat, tel_bat)

              tel_predictions = transformer.call(eng_bat, tel_bat,
                                          encoder_self_attention_mask,
                                          decoder_self_attention_mask,
                                          decoder_cross_attention_mask,
                                          enc_start_token = False,
                                          enc_end_token = False,
                                          dec_start_token = True,
                                          dec_end_token = True)
              labels = transformer.decoder.sentence_embedding.batch_tokenize(tel_bat, start_token=True, end_token=True)
              loss = calculate_loss(tel_predictions, labels, tel_to_ix[PADDING_TOKEN])
          gradients = tape.gradient(loss, transformer.trainable_variables)
          optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

          if batch_num % 500 == 0:
              print(f"Iteration {batch_num} : {loss.numpy()}")
              print(f"English: {eng_bat[0]}")
              print(f"Telugu Translation: {tel_bat[0]}")
              tel_sentence_predicted = tf.argmax(tel_predictions[0], axis=1)
              predicted_sentence = ""
              for idx in tel_sentence_predicted:
                if idx == tel_to_ix[END_TOKEN]:
                  break
                predicted_sentence += ix_to_tel[idx.numpy()]
              print(f"Telugu Prediction: {predicted_sentence}")

              tel_sen = ("",)
              eng_sen = ["should we go to mall?",]
              for word_counter in range(max_sequence_length):
                  encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_mask(eng_sen, tel_sen)
                  predictions = transformer.call(eng_sen, tel_sen,
                                          encoder_self_attention_mask,
                                          decoder_self_attention_mask,
                                          decoder_cross_attention_mask,
                                          enc_start_token = False,
                                          enc_end_token = False,
                                          dec_start_token = True,
                                          dec_end_token = False)
                  next_token_prob_distribution = predictions[0][word_counter]
                  next_token_index = tf.argmax(next_token_prob_distribution).numpy()
                  next_token = ix_to_tel[next_token_index]
                  tel_sen = (tel_sen[0] +  next_token,)
                  if next_token == END_TOKEN: 
                      break
              print(f"Evaluation translation (should we go to the mall?) : {tel_sen}")
              print("-------------------------------------------")

Epoch 0
Iteration 0 : 1.8223562240600586
English: these books were published in bengali, english, hindi, urdu, and nepali language.
Telugu Translation: హిందీ, ఆంగ్లం, తెలుగు, ఉర్దూ, పర్షియా భాషల పుస్తకాలు డిజిటైజ్ చేసారు.
Telugu Prediction: నంణణినననీీనననటణణణణణణణణంంిిిిిిిిిిింంిిిిిింంననయననీీీీీయయాాాంంంమమమమమంంంంంంంీీణణణణణణణణణణణణణణణ్్్ణణణణణణ్్ిిిి్్్ణణణణణణణణణణణణణణణణణణణణణణణణవణణణణణటుుుంణణణణణణణణణణిిిణణణణణణణణణణణణణణాాాణణణణణటటటిాాాాాుణణణుుుుిిి
Evaluation translation (should we go to the mall?) : ('ంంంంానననీీనటటటటటటటంంంంంంంంిిింంంంంంంంంంంంంంంంంంయయీీీీీీీంంంంంంంంంమంంంంంంంంంంంంంంంంంంంంంంంంం్్్ంంంణణంంంం్్్్్్్్్ణణంంంణణణణాాాాాణణణణ్్ంంంంంంంటటటుంంంంణణణంంంిిిిిిణణణణణణణణణణణాాాాాాాణణాాటటాాాాాాుుుుుుుుిిి',)
-------------------------------------------
Iteration 500 : 1.669479489326477
English: we are running at 7.5% per year, but will grow even faster in the coming years.
Telugu Translation: ఇప్పుడు మనం ఏడాదికి 7.5 శాతం వృద్ధి రేటుతో ముందుకు వెళుతున్నాం.
Telugu Prediction: నయణణిిననణణనననణణణణణణణణణసిిిిిి